In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
! pip install impyute

In [ ]:
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
import lightgbm as lgb
from sklearn.metrics import *
from sklearn.model_selection import KFold

In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')

In [ ]:
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

In [ ]:
train_quality_df = train_quality.drop(['time','fwver'],axis=1)
train_quality_df.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [ ]:
train_quality_df.isnull().sum()

user_id            0
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [ ]:
# train_quality 결측치 최빈값 0으로 대체 
train_quality_df.fillna(0,inplace=True)
train_quality_df.isnull().sum()

user_id       0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

In [ ]:
train_quality_df.dtypes

user_id         int64
quality_0     float64
quality_1       int64
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object

In [ ]:
def string2num(x):
  x = re.sub(r"[^0-9]+",'',str(x))
  if x == '':
    return 0
  else:
    return int(x)

In [ ]:
train_quality_df['quality_5'] = train_quality_df['quality_5'].apply(lambda x : string2num(x))
train_quality_df['quality_7'] = train_quality_df['quality_7'].apply(lambda x : string2num(x))
train_quality_df['quality_8'] = train_quality_df['quality_8'].apply(lambda x : string2num(x))
train_quality_df['quality_9'] = train_quality_df['quality_9'].apply(lambda x : string2num(x))
train_quality_df['quality_10'] = train_quality_df['quality_10'].apply(lambda x : string2num(x))

In [ ]:
train_quality_df['quality_5'] = train_quality_df['quality_5'].astype('int')
train_quality_df['quality_7'] = train_quality_df['quality_7'].astype('int')
train_quality_df['quality_8'] = train_quality_df['quality_8'].astype('int')
train_quality_df['quality_9'] = train_quality_df['quality_9'].astype('int')
train_quality_df['quality_10'] = train_quality_df['quality_10'].astype('int')

In [ ]:
mean_train_quality = train_quality_df.groupby('user_id').agg('mean')
mean_train_quality = mean_train_quality.reset_index()
mean_train_quality

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.000,6.000000,0.000000,0.0
1,10002,0.000000,-0.020833,-0.010417,0.0,0.0,0.343750,0.458333,5.750000,0.0,0.125,3.875000,-0.020833,0.0
2,10004,-0.083333,-0.083333,-0.083333,0.0,0.0,0.250000,3.541667,43.500000,0.0,0.000,2.000000,-0.083333,0.0
3,10005,-0.416667,-0.416667,-0.416667,0.0,0.0,0.833333,1.083333,18.000000,0.0,0.000,5.000000,-0.416667,0.0
4,10006,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.111111,1.333333,0.0,0.000,4.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,0.000000,-0.333333,-0.333333,0.0,0.0,0.416667,-0.333333,0.000000,0.0,0.000,1.000000,-0.333333,0.0
8277,24992,-0.333333,-0.333333,-0.333333,0.0,0.0,0.861111,-0.333333,0.000000,0.0,0.000,6.333333,-0.333333,0.0
8278,24993,-0.291667,-0.291667,-0.291667,0.0,0.0,0.625000,-0.291667,0.000000,0.0,0.000,4.000000,-0.291667,0.0
8279,24995,-0.416667,-0.416667,-0.416667,0.0,0.0,0.625000,-0.416667,0.000000,0.0,0.000,2.500000,-0.416667,0.0


In [ ]:
mean_train_quality.describe()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,8281.000000,8281.000000,8281.000000,8281.000000,8281.0,8281.0,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000
mean,17515.508996,2.976648,-0.188322,3.246467,0.0,0.0,63.950054,2.178745,28.563257,0.158620,41.357528,764.982573,-0.200108,0.017214
std,4322.740630,101.969441,0.219599,102.651155,0.0,0.0,1035.677913,18.066568,216.655614,0.901582,1231.877870,12428.098464,0.210423,0.140654
min,10000.000000,-0.916667,-0.916667,-0.916667,0.0,0.0,0.000000,-0.916667,0.000000,0.000000,0.000000,0.000000,-0.916667,0.000000
25%,13763.000000,-0.305556,-0.309524,-0.321429,0.0,0.0,0.333333,-0.256410,0.000000,0.000000,0.000000,2.500000,-0.333333,0.000000
50%,17539.000000,-0.125000,-0.138889,-0.145833,0.0,0.0,0.611111,-0.083333,0.000000,0.000000,0.000000,4.000000,-0.152542,0.000000
75%,21282.000000,0.000000,0.000000,0.000000,0.0,0.0,1.277778,0.000000,0.214286,0.000000,0.000000,13.000000,0.000000,0.000000
max,24997.000000,5409.750000,3.797619,5409.750000,0.0,0.0,57637.750000,464.666667,5576.000000,47.071429,64928.000000,691646.000000,0.333333,4.571429


In [ ]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-10000,err-1]+=1
error.shape

100%|██████████| 16554663/16554663 [00:47<00:00, 349473.94it/s]


(15000, 42)

In [ ]:
df_error = pd.DataFrame(error)
df_error['user_id'] = train_err['user_id']
df_error

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,user_id
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,10000
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,10000
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,10011
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10011
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,10011
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,10011


In [ ]:
len(df_error)

15000

In [ ]:
merged_train_err_quality = pd.merge(df_error,mean_train_quality,on='user_id',how='left')
del merged_train_err_quality['user_id']
merged_train_err_quality.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [ ]:
len(merged_train_err_quality)

15000

In [ ]:
multiple_imputation_df = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_train_err_quality))
multiple_imputation_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.740000,0.003769,2.499384,-0.592755,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.739999,0.003768,2.499384,-0.592754,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.740000,0.003768,2.499384,-0.592755,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.740000,0.003768,2.499384,-0.592755,0.0


In [ ]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [ ]:
train_x = multiple_imputation_df.values
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 55)
(15000,)


In [ ]:
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 3 rounds.
[20]	valid_0's auc: 0.79739	valid_0's pr_auc: 0.794701
Early stopping, best iteration is:
[32]	valid_0's auc: 0.803129	valid_0's pr_auc: 0.799819
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.78607	valid_0's pr_auc: 0.318309
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.758098	valid_0's pr_auc: 0.35982
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[12]	valid_0's auc: 0.808057	valid_0's pr_auc: 0.55147
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[10]	valid_0's auc: 0.800269	valid_0's pr_auc: 0.974185


In [ ]:
print(np.mean(auc_scores))

0.7911245708229885


In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:47<00:00, 345119.89it/s]

(14999, 42)


In [ ]:
test_quality_df = test_quality.drop(['time','fwver'],axis=1)

In [ ]:
test_quality_df.isnull().sum()

user_id            0
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [ ]:
test_quality_df.dtypes

user_id         int64
quality_0     float64
quality_1      object
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object

In [ ]:
test_quality_df['quality_1'] = test_quality_df['quality_1'].apply(lambda x : string2num(x))
test_quality_df['quality_5'] = test_quality_df['quality_5'].apply(lambda x : string2num(x))
test_quality_df['quality_7'] = test_quality_df['quality_7'].apply(lambda x : string2num(x))
test_quality_df['quality_8'] = test_quality_df['quality_8'].apply(lambda x : string2num(x))
test_quality_df['quality_9'] = test_quality_df['quality_9'].apply(lambda x : string2num(x))
test_quality_df['quality_10'] = test_quality_df['quality_10'].apply(lambda x : string2num(x))

In [ ]:
test_quality_df['quality_1'] = test_quality_df['quality_1'].astype('int')
test_quality_df['quality_5'] = test_quality_df['quality_5'].astype('int')
test_quality_df['quality_7'] = test_quality_df['quality_7'].astype('int')
test_quality_df['quality_8'] = test_quality_df['quality_8'].astype('int')
test_quality_df['quality_9'] = test_quality_df['quality_9'].astype('int')
test_quality_df['quality_10'] = test_quality_df['quality_10'].astype('int')

In [ ]:
test_quality_df.fillna(0,inplace=True)
test_quality_df.isnull().sum()

user_id       0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

In [ ]:
test_quality_df.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [ ]:
mean_test_quality = test_quality_df.groupby('user_id').agg('mean')
mean_test_quality = mean_test_quality.reset_index()
mean_test_quality

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.00,0.0,0.00,2.000000,0.000000,0.0
1,30001,-0.250000,0.250000,-0.250000,0.0,0.0,71.444444,-0.250000,0.00,0.0,0.00,854.333333,-0.250000,0.0
2,30002,-0.125000,0.125000,-0.125000,0.0,0.0,1.233333,-0.125000,0.00,0.0,0.00,13.300000,-0.125000,0.0
3,30003,0.000000,0.000000,0.000000,0.0,0.0,0.200000,3.666667,44.00,0.0,0.00,2.400000,0.000000,0.0
4,30004,-0.250000,0.250000,-0.250000,0.0,0.0,0.333333,1.125000,16.50,0.0,0.00,1.000000,-0.250000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,44990,-0.111111,0.111111,-0.111111,0.0,0.0,1.500000,-0.111111,0.00,0.0,0.00,16.666667,-0.111111,0.0
8264,44993,0.145833,0.041667,0.145833,0.0,0.0,0.333333,2.229167,27.25,0.0,2.25,3.500000,-0.041667,0.0
8265,44994,-0.916667,0.916667,-0.916667,0.0,0.0,1.083333,-0.916667,0.00,0.0,0.00,2.000000,-0.916667,0.0
8266,44996,-0.046296,0.046296,-0.046296,0.0,0.0,0.388889,-0.046296,0.00,0.0,0.00,4.111111,-0.046296,0.0


In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:47<00:00, 350778.18it/s]

(14999, 42)


In [ ]:
df_error_test = pd.DataFrame(test_x)
df_error_test['user_id'] = test_err['user_id']
df_error_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,user_id
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,30000
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30000
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,30000
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,30000
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,30015
14995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,30015
14996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,30015
14997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,0.0,1.0,0.0,11127.0,13161.0,0.0,3.0,44.0,0.0,0.0,0.0,0.0,78.0,0.0,14.0,0.0,0.0,1.0,1.0,3.0,0.0,30.0,49.0,0.0,30015


In [ ]:
merged_test_err_quality = pd.merge(df_error_test,mean_test_quality,on='user_id',how='left')
del merged_test_err_quality['user_id']
merged_test_err_quality.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0


In [ ]:
len(merged_test_err_quality)

14999

In [ ]:
merged_test_err_quality.isnull().sum()

0                0
1                0
2                0
3                0
4                0
5                0
6                0
7                0
8                0
9                0
10               0
11               0
12               0
13               0
14               0
15               0
16               0
17               0
18               0
19               0
20               0
21               0
22               0
23               0
24               0
25               0
26               0
27               0
28               0
29               0
30               0
31               0
32               0
33               0
34               0
35               0
36               0
37               0
38               0
39               0
40               0
41               0
quality_0     2292
quality_1     2292
quality_2     2292
quality_3     2292
quality_4     2292
quality_5     2292
quality_6     2292
quality_7     2292
quality_8     2292
quality_9     2292
quality_10  

In [ ]:
multiple_imputation_df_test = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_test_err_quality))
multiple_imputation_df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,0.0,1.0,0.0,11127.0,13161.0,0.0,3.0,44.0,0.0,0.0,0.0,0.0,78.0,0.0,14.0,0.0,0.0,1.0,1.0,3.0,0.0,30.0,49.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
pred_y_list = []
for model in models:
    pred_y = model.predict(multiple_imputation_df_test.values)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[0.57688764],
       [0.27087552],
       [0.31913551],
       ...,
       [0.39582502],
       [0.6160444 ],
       [0.33976674]])

In [ ]:
pred = np.where(pred_ensemble>0.35,1,0)
pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
sample_submission['problem'] = pred
sample_submission.head()

,user_id,problem
0,30000,1
1,30001,0
2,30002,0
3,30003,1
4,30004,1


In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/lgb_baseline_merged_err_quality_imputation_submission_threshold0.35.csv')